In [25]:
import baostock as bs
import pandas as pd
import numpy as np
import time
import os

## Introduction

This notebook demonstrates how to calculate the realized volatility of the constituent stocks of the HS300 index. The process involves several steps, starting with fetching the constituent stocks using the Baostock API.

We would like to thank [Baostock](http://baostock.com/baostock/index.php/%E9%A6%96%E9%A1%B5) for providing the data used in this research. And to facilitate the readers, we have organized the code into functions. You can simply call these functions to execute the respective tasks.

### Get constituen stocks of HS300 index
We obtain the constituent stocks of the HS300 index by using the Baostock API at the date of acquisition.

In [26]:
def get_constituen_stocks():
    lg = bs.login()

    rs = bs.query_hs300_stocks(date='2024-1-6')

    # get hs300 constituent stocks
    hs300_stocks = []
    while (rs.error_code == '0') & rs.next():
        hs300_stocks.append(rs.get_row_data())
    hs300 = pd.DataFrame(hs300_stocks, columns=rs.fields)
    
    bs.logout()
    
    hs300.to_csv('hs300_stocks.csv', index=True)
    return hs300

# get_constituen_stocks()

### Complete Stock Data Information

After obtaining the constituent stocks of the HS300 index, we complete the dataset by adding additional information such as:

- Abbreviated Symbol
- Sector Code
- Sector Code Name
- Industry Code
- Industry Name

In [27]:
def code_order_id(code):
    if code[:3] == 'sh.':
        return code[3:] + '.XSHG'
    elif code[:3] == 'sz.':
        return code[3:] + '.XSHE'
    else:
        Warning('code is not valid')
        return None

# add abbrev_symbol, sector_code, sector_code_name, industry_code, industry_name
def add_extra_info():
    hs300_stocks = pd.read_csv('hs300_stocks.csv', index_col=0)
    all_stocks = pd.read_csv('overall_description.csv')

    for code in hs300_stocks['code']:
        order_id = code_order_id(code)
        stock_info = all_stocks[all_stocks['order_book_id'] == order_id]
        if len(stock_info) == 0:
            print(f'stock info not found for {code}/{order_id}')
            continue
        hs300_stocks.loc[hs300_stocks['code'] == code, 'abbrev_symbol'] = stock_info['abbrev_symbol'].item()
        hs300_stocks.loc[hs300_stocks['code'] == code, 'sector_code'] = stock_info['sector_code'].item()
        hs300_stocks.loc[hs300_stocks['code'] == code, 'sector_code_name'] = stock_info['sector_code_name'].item()
        hs300_stocks.loc[hs300_stocks['code'] == code, 'industry_code'] = stock_info['industry_code'].item()
        hs300_stocks.loc[hs300_stocks['code'] == code, 'industry_name'] = stock_info['industry_name'].item()
        
    hs300_stocks.to_csv('hs300_stocks.csv', index=True)

def group_by_industry():
    hs300_stocks = pd.read_csv('hs300_stocks.csv', index_col=0)
    grouped = hs300_stocks.sort_values('sector_code').reset_index()
    grouped = grouped.rename(columns={'index' : 'original_index'})
    grouped.to_csv('hs300_stocks.csv', index=True)

# add_extra_info()
# group_by_industry()

### Acquire High-Frequency Data

To analyze the high-frequency trading patterns of each stock in the HS300 index, we first acquire 5-minute interval data. This data is saved in the `/5mins_data/code.csv` format for each stock.

1. **Fetching Data**: Use the Baostock API to fetch the 5-minute interval data.
2. **Saving Data**: Save the data in the specified directory.
3. **Data Cleaning and Processing**: Perform necessary data cleaning and processing to ensure the data is ready for analysis.

*Note: To obtain daily historical data instead of 5-minute intervals, simply change the `frequency` parameter to `'d'` in the `bs.query_history_k_data_plus` function.*

In [28]:
def get_stock_data(code):
    start_time = time.time()
    rs = bs.query_history_k_data_plus(code,
        "date, time, code, open, high, low, close, volume",
        start_date='2006-01-01', end_date='2023-12-31',
        frequency="5", adjustflag="1")
    print(f'recived data for {code}, time: {(time.time() - start_time):.1f}')
    
    start_time = time.time()
    data_list = []
    while (rs.error_code == '0') & rs.next():
        data_list.append(rs.get_row_data())
    result = pd.DataFrame(data_list, columns=rs.fields)
    result.to_csv(f'./5mins_data/{code}.csv', index=False)
    print(f'convert data to dataframe, time: {(time.time() - start_time):.1f}')
    
    return

def get_data():
    hs300 = pd.read_csv('hs300_stocks.csv', index_col=0)
    bs.login()

    code_list = hs300['code']
    for i, code in enumerate(code_list):
        print(f'get data for {code} ({i + 1}/300)')
        if os.path.exists(f'./5mins_data/{code}.csv'):
            print(f'{code} already exists, skip')
            continue
        get_stock_data(code)

    bs.logout()

# get_data()

# change the time format from 20190101093000000000 to 09:30:00
def reformulate_time(code):
    stock_data = pd.read_csv(f'./5mins_data/{code}.csv')
    stock_data['time'] = pd.to_datetime(stock_data['time'], format='%Y%m%d%H%M%S%f')
    stock_data['time'] = stock_data['time'].dt.strftime('%H:%M')
    stock_data.to_csv(f'./5mins_data/{code}.csv', index=False)

# add start_date and end_date of trading to hs300_stocks.csv
# reformulate the time format of 5 mins data
def add_date_info():
    hs300 = pd.read_csv('hs300_stocks.csv', index_col=0)
    for code in hs300['code']:
        reformulate_time(code)
        stock_data = pd.read_csv(f'./5mins_data/{code}.csv')
        start_date = stock_data['date'].iloc[0]
        end_date = stock_data['date'].iloc[-1]
        hs300.loc[hs300['code'] == code, 'start_date'] = start_date
        hs300.loc[hs300['code'] == code, 'end_date'] = end_date
    hs300.to_csv('hs300_stocks.csv')

# add_date_info()

### RV Calculation
We calculate the realized volatility for each stock, and then combine it into a single dataframe (/realized_volatility/rv_table.csv) with index to be unique date and columns to be stock code.

In [30]:
# calculate the realized volatility by intra-day data
def cal_rv_daily(intraday_close):
    if len(intraday_close) == 0:
        return 0
    if len(intraday_close) != 44:
        return 0
    log_intra_price = np.log(intraday_close)
    log_intra_return = np.diff(log_intra_price) * 100 # percentage
    rv = np.sum(log_intra_return ** 2)
    return rv

# for each stock, calculate the realized volatility of each day
def cal_rv_stock(code):
    stock_data = pd.read_csv(f'./5mins_data/{code}.csv')
    date_list = stock_data['date'].unique()
    rv_list = []
    for date in date_list:
        filter1 = stock_data['date'] == date
        # 09:55 is used to calculate the rv of 10:00
        filter2 = stock_data['time'] >= '09:55'
        intraday_close = stock_data[filter1 & filter2]['close']
        if len(intraday_close) != 43:
            Warning(f'code: {code}, date: {date}, there may exist missing data')
        rv = cal_rv_daily(intraday_close)
        rv_list.append(rv)
    
    return date_list, rv_list

# quick method to calculate the realized volatility of each day
def cal_rv_stock_quick(code):
    stock_data = pd.read_csv(f'./5mins_data/{code}.csv')
    stock_data['log_return'] = np.log(stock_data['close']) - np.log(stock_data['close'].shift(1))
    date_list = stock_data['date'].unique()
    rv_list = []
    filter_time = stock_data[stock_data['time'] >= '10:00']
    for date in date_list:
        filter1 = filter_time['date'] == date
        log_returns = filter_time[filter1]['log_return'] * 100 # percentage
        if len(log_returns) != 43:
            Warning(f'code: {code}, date: {date}, there may exist missing data')
        rv = np.sum(log_returns ** 2)
        rv_list.append(rv)
    
    return date_list, rv_list

def cal_rv_all():
    hs300 = pd.read_csv('hs300_stocks.csv', index_col=0)
    rv_columns = []
    
    for i, code in enumerate(hs300['code']):
        start_time = time.time()
        date_list, rv_list = cal_rv_stock_quick(code)
        rv_column = pd.DataFrame(rv_list, index=date_list, columns=[code])
        rv_columns.append(rv_column)
        print(f'{code} ({i + 1}/300) calculated, time used: {(time.time() - start_time):.1f}')

    rv_table = pd.concat(rv_columns, axis=1)
    rv_table = rv_table.sort_index()
    rv_table.to_csv('realized_volatility/rv_table.csv')

# cal_rv_all()

### Double Check the Data

It's essential to verify the integrity of the data in the `rv_table`. Specifically, we need to ensure that any `NaN` values in the `rv_table` are due to non-trading days. This step helps in maintaining the accuracy of our analysis and ensures that the missing data points are not due to errors in data acquisition.

In [31]:
# get the trading status stocks of each day
def get_tradestatus(code, start_date, end_date):
    bs.login()

    rs = bs.query_history_k_data_plus(code,
        "date, code, tradestatus, open, high, low, close, volume",
        start_date=start_date, end_date=end_date,
        frequency="d", adjustflag="1")

    data_list = []
    while (rs.error_code == '0') & rs.next():
        data_list.append(rs.get_row_data())
    result = pd.DataFrame(data_list, columns=rs.fields)
    result.to_csv(f'./daily_data/{code}.csv', index=False)
    bs.logout()

    return result

# check the consistency of the trading status and the realized volatility
def double_check(code, na_dates_in_rv_table):
    not_consistent = []
    daily_data = pd.read_csv(f'./daily_data/{code}.csv', index_col=0)
    non_trading_dates = daily_data[daily_data['tradestatus'] == 0].index
    na_dates_in_rv_table = na_dates_in_rv_table[(na_dates_in_rv_table >= daily_data.index[0]) & (na_dates_in_rv_table <= daily_data.index[-1])]
    
    na_but_trading = [date for date in na_dates_in_rv_table if date not in non_trading_dates]
    na_but_trading_copy = na_but_trading.copy()
    non_trading_but_not_na = [date for date in non_trading_dates if date not in na_dates_in_rv_table]
    for date in na_but_trading_copy:
        if date not in daily_data.index:
            na_but_trading.remove(date)
    
    if len(na_but_trading) != 0:
        print(f'code: {code}, na but trading: {na_but_trading}')
    if len(non_trading_but_not_na) != 0:
        print(f'code: {code}, non trading but not na: {non_trading_but_not_na}')
    
    not_consistent = na_but_trading + non_trading_but_not_na
    
    return not_consistent

def check_all_date():
    rv_table = pd.read_csv('realized_volatility/rv_table.csv', index_col=0)
    hs300_stocks = pd.read_csv('hs300_stocks.csv', index_col=0)
    code_list = rv_table.columns

    for code in code_list:
        start_date = hs300_stocks[hs300_stocks['code'] == code]['start_date'].iloc[0]
        end_date = hs300_stocks[hs300_stocks['code'] == code]['end_date'].iloc[0]
        na_dates = rv_table[rv_table[code].isna()].index
        na_dates = na_dates[(na_dates >= start_date) & (na_dates <= end_date)]
        # print(na_dates)
        get_tradestatus(code, start_date, end_date)
        
        # not_consistent = double_check(code, na_dates)
        # if len(not_consistent) != 0:
        #     print(f'double check of code: {code} finished, not consistent dates are : {not_consistent}')
        # break
        
# check_all_date()

### Calculate Daily Log Returns

The next step is to calculate the daily log returns for each stock.

In [32]:
def cal_return_stock(code):
    stock_data = pd.read_csv(f'./daily_data/{code}.csv', index_col=0)
    log_return = np.log(stock_data['close']) - np.log(stock_data['close'].shift(1))
    return stock_data.index, (log_return * 100).tolist() # percentage

def cal_return_all():
    hs300 = pd.read_csv('hs300_stocks.csv', index_col=0)
    return_columns = []
    
    for i, code in enumerate(hs300['code']):
        date_list, return_list = cal_return_stock(code)
        return_column = pd.DataFrame(return_list, index=date_list, columns=[code])
        return_columns.append(return_column)

    return_table = pd.concat(return_columns, axis=1)
    return_table = return_table.sort_index()
    return_table.fillna(0, inplace=True)
    return_table.to_csv('return_table.csv')

# cal_return_all()